(sec:correlation)=
# Electron correlation

## Two-particle density

The electron correlation describes instantaneous interaction between electrons in a molecular system. This interaction manifests itself as a relationship between one- and two-electron density matrices 

$$
\rho(\mathbf{r}, \mathbf{r}') \neq \rho(\mathbf{r}) \rho(\mathbf{r}')
$$

where $\rho(\mathbf{r}, \mathbf{r}')$ is the two-electron density. If the molecular system is described by the wave function model, which does not permit instantaneous electron interaction, the relationship between density matrices becomes 

$$
\rho(\mathbf{r}, \mathbf{r}') = \rho(\mathbf{r}) \rho(\mathbf{r}')
$$

indicating completely independent motion of electron in the molecular system.

## Correlation energy

The electron correlation arises from two primary sources namely the fermionic nature of electrons and the interaction between electrons. From the perspective of the Hartree–Fock method the first but not the second source is accounted for, and as a measure of this discrepancy the correlation energy is defined as

$$
E_{\textrm{corr}} = E_{\textrm{exact}} - E_{\textrm{HF}} 
$$

In work with approximate methods such as the plethora of wave function models, it is convenient to introduce a method specific correlation energy following Löwdin {cite:p}`Lowdin1980`

$$
E_{\textrm{corr}} = E_{\textrm{WF}} - E_{\textrm{HF}}  
$$

where $E_{\textrm{WF}}$ is the energy of the molecular system computed by the selected wave function model.

To illustrate these concepts, we will use the configuration interaction (CI) method to study the H$_2$ molecule. The obtained results provide the exact correlation energy within the chosen basis set.

In [1]:
import veloxchem as vlx
import multipsi as mtp
import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate
from scipy.integrate import simps

mol_str = """
H     0.000000    0.000000   -0.370500
H     0.000000    0.000000    0.370500
"""

au_to_nm = 0.0529177 # length conversion factor
Hz = 0.370500 / 10 / au_to_nm #Position of the H in atomic units.

molecule = vlx.Molecule.read_str(mol_str, units='angstrom')
basis = vlx.MolecularBasis.read(molecule, 'sto-3g')

scf_drv = vlx.ScfRestrictedDriver()
scf_drv.compute(molecule, basis)

space=mtp.OrbSpace(molecule,scf_drv.mol_orbs)
space.FCI()
CIdrv=mtp.CIDriver()
CIdrv.compute(molecule,basis,space)

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 8.
                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error

TypeError: __init__() missing 3 required positional arguments: 'molecule', 'basis', and 'molspace'

If we print the HF and the CI energy, we can see that the CI energy is indeed lower, although the electron correlation is a relatively small correction compared to the absolute energy of the molecule. In other words, HF already captured by far most of the energy of the system.

In [ ]:
print("Hartree-Fock energy:             ",scf_drv.get_scf_energy())
print("Configuration interaction energy:",CIdrv.getEnergy(0))
print("Correlation energy:              ", CIdrv.getEnergy(0)-scf_drv.get_scf_energy())

## Visualizing electron correlation

### One- and two-particle densities

Now that we have both the Hartree–Fock and a correlated wavefunction, we can visualize the densities using the `VisualizationDriver` class in Veloxchem. This module will give us the value of the orbitals of the system (here the $\sigma_g$ and $\sigma_u$) along the z-axis.

In [ ]:
def get_orbs(molecule, basis, mol_orbs, coords):
    # visualization
    vis_drv = vlx.VisualizationDriver()

    # mo values at given coordinates
    mol_orbs = scf_drv.mol_orbs

    nMO=basis.get_dimensions_of_basis(molecule)

    orbs=[]
    for i in range(nMO):
        orbs.append(np.array(vis_drv.get_mo(coords, molecule, basis, mol_orbs, i, 'alpha')))
    return orbs

We can now plot the densities of both HF and CI along this axis. The density is simply the product of the density matrix with the value of the density of the orbitals on the grid. For HF, the density matrix is simply "2" for the occupied orbitals and "0" for the unoccupied ones. For CI, it is a bit more complicated, but is provided by the CI driver.

In [ ]:
CIDM=CIdrv.get1den(0)
print("CI density matrix")
print(CIDM)

In [ ]:
def get_n1(orbs, CIDM):
    nMO = len(orbs)
    
    #HF density for a 2-electron system
    n1_HF = 2 * orbs[0]**2
    
    #CI Density
    n1_CI = np.zeros(n)
    for i in range(nMO):
        for j in range(nMO):
            n1_CI += CIDM[i,j] * orbs[i] * orbs [j]

    return n1_HF, n1_CI

In [ ]:
n = 100
coords = np.zeros((n,3))
coords[:,2] = np.linspace(-3, 3, n, endpoint=True)

orbs = get_orbs(molecule, basis, scf_drv.mol_orbs, coords)

n1_HF, n1_CI = get_n1(orbs,CIDM)

fig = plt.figure(1)
plt.axes()
plt.plot(coords[:,2], n1_HF, label='HF')
plt.plot(coords[:,2], n1_CI, label='CI')
plt.legend()
plt.axvline(x=Hz, color='0.5', linewidth = 0.7, dashes = [3,1,3,1])
plt.axvline(x=-Hz, color='0.5', linewidth = 0.7, dashes = [3,1,3,1])
plt.setp(plt.gca(), xlim = (-3,3), ylim = (0,0.4))

As you can see, the difference in densities is very minor. Looking closely, it is possible to notice a little decreased density in the middle of the bond for CI, compensated by a minor increase around the nuclei. It fits the intuitive picture that correlation allows the electrons to stay away from each other, increasing the probability that each is around a different nucleus and reducing the probability that they both stand in the center of the bond. Still the effect is relatively minor.

Let us now look instead at the two-body density:

In [ ]:
def get_n12(orbs, orbs2, CIDM2):
    nMO = len(orbs)
    
    #HF 2-body density
    n12_HF = 2 * orbs2[0]**2 * orbs[0]**2
    
    #CI 2-body density
    n12_CI = np.zeros(n)
    for i in range(nMO):
        for j in range(nMO):
            for k in range(nMO):
                for l in range(nMO):
                    n12_CI += CIDM2[i,j,k,l] * orbs[i] * orbs[j] * orbs2[k] * orbs2[l]

    return n12_HF, n12_CI

In [ ]:
CIDM2=CIdrv.get2den(0)
print("CI 2-body density matrix")
print(CIDM2)

For this example we are fixing the position of the first electron to be on the nucleus:

In [ ]:
# electron 1 at the position of the hydrogen nucleus
# electron 2 anywhere on the internuclear axis
h1 = [[0, 0, Hz]]
orbs2 = get_orbs(molecule, basis, scf_drv.mol_orbs, h1)
n12_HF, n12_CI = get_n12(orbs,orbs2, CIDM2)

fig = plt.figure(2)
plt.axes()
plt.plot(coords[:,2], n12_HF)
plt.plot(coords[:,2], n12_CI)

plt.axvline(x=Hz, color='0.5', linewidth = 0.7, dashes = [3,1,3,1])
plt.axvline(x=-Hz, color='0.5', linewidth = 0.7, dashes = [3,1,3,1])
plt.setp(plt.gca(), xlim = (-3,3), ylim = (0,0.1))

This results shows very clearly how for the CI wavefunction, the density of electron 2 for electron 1 fixed at the hydrogen atom is very different than the total density, with the second electron preferentially sitting on the other hydrogen. This effect was not showing in the 1-body density because the 1-body density shows only the averaged result, taking into account all possible positions of the other electron. On the other hand, for the Hartree-Fock wavefunction, even the two-body density does not show any correlation: the wavefunction itself is uncorrelated.

### Electron cusp and basis set convergence

As we have seen, the point charge of the nucleus creates a cusp (*i.e.* a discontinuity in the derivative) in the electronic wavefunction near the nucleus, clearly seen in the Slater type orbitals. Similarly, the point charge of a fixed electron should create a cusp in the wavefunction of the other electrons. However, in standard quantum chemistry methods, the wavefunction does not contain any explicit term depending on the interelectronic distance. In other words, our n-electron basis (the Slater Determinants) is very ill-suited to describe this cusp and we need a lot of functions to accurately describe it.

Let's illustrate this concept on Helium, using configuration interaction with an increasing basis set size (and thus also an increasing number of determinants). This time we are going to move the first electron in a circle around the atom.

In [ ]:
He_str = """
He     0.000000    0.000000   0.000000
"""
Helium = vlx.Molecule.read_str(He_str, units='angstrom')

#First electron in a circle around the atom
theta = np.linspace(-np.pi, np.pi, n, endpoint=True)

coords = np.zeros((n,3))
coords[:,0] = Hz * np.cos(theta)
coords[:,1] = Hz * np.sin(theta)

#Second electron fixed
coords2 = [[Hz, 0, 0]]

#Save the 1 and 2-body densities for each basis
n1_HFs=[]
n12_HFs=[]
n12_CIs=[]

for basis_lbl in ["sto-3g", 'cc-pvdz', 'cc-pvtz']:
    #Compute CI
    basis = vlx.MolecularBasis.read(Helium, basis_lbl)
    scf_drv.compute(Helium, basis)
    space=mtp.OrbSpace(Helium,scf_drv.mol_orbs)
    space.FCI()
    CIdrv=mtp.CIDriver()
    CIdrv.compute(Helium,basis,space)

    #Get the orbitals
    orbs = get_orbs(Helium, basis, scf_drv.mol_orbs, coords)
    orbs2 = get_orbs(Helium, basis, scf_drv.mol_orbs, coords2)

    #Get the densities
    n1_HF, n1_CI = get_n1(orbs,CIdrv.get1den(0))
    n1_HFs.append(n1_HF)
    
    n12_HF, n12_CI = get_n12(orbs,orbs2, CIdrv.get2den(0))
    n12_HFs.append(n12_HF)
    n12_CIs.append(n12_CI)

In [ ]:
fig = plt.figure(4)
plt.axes()
plt.plot(theta, n12_CIs[0], label='STO-3G')
plt.plot(theta, n12_CIs[1], label='cc-pVDZ')
plt.plot(theta, n12_CIs[2], label='cc-pVTZ')
plt.legend()
plt.axvline(x=Hz, color='0.5', linewidth = 0.7, dashes = [3,1,3,1])
plt.axvline(x=-Hz, color='0.5', linewidth = 0.7, dashes = [3,1,3,1])
plt.setp(plt.gca(), xlim = (-3,3), ylim = (0.02,0.05))

The basis set difference here is quite significant. With STO-3G, a minimal basis, there is no virtual orbitals and thus no correlation. As we increase the basis set size, the correlation hole gets sharper, but even with a cc-pVTZ, the hole does not look like a cusp but is still very slow-varying. This illustrates the reason why correlation methods require larger basis to converge, as opposed to Hartree-Fock or Density Functional Theory where the basis set only needs to reproduce the 1-body density accurately, which converges much faster.